In [71]:
import pandas as pd
import numpy as np

In [72]:
train = pd.read_csv('train.csv')
train['DepartmentDescription'] = train['DepartmentDescription'].replace('MENSWEAR','MENS WEAR')
train_df = train.copy()
testing_df = train_df.copy()
test = pd.read_csv('test.csv')

In [73]:
testing_df.isna().sum()

TripType                    0
VisitNumber                 0
Weekday                     0
Upc                      4129
ScanCount                   0
DepartmentDescription    1361
FinelineNumber           4129
dtype: int64

## Missing in DepartmentDescription

In [74]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_df['DepartmentDescription'] = le.fit_transform(list(train_df['DepartmentDescription']))

In [75]:
train_null_dd = train_df[train_df['DepartmentDescription']==67]
train_null_dd = train_null_dd.groupby('VisitNumber',as_index=False)['ScanCount'].count()

In [76]:
train_null_dd.rename(columns={'ScanCount': 'Null'}, inplace=True)

In [77]:
train_null_dd.tail()

,VisitNumber,Null
1167,190230,1
1168,190309,1
1169,190408,1
1170,190651,1
1171,191080,3


In [78]:
train_null_dd.shape

(1172, 2)

In [79]:
testing_df = testing_df.merge(train_null_dd,how='left',on='VisitNumber',copy=True)
testing_df['Null'].fillna(1,inplace=True)
testing_df['Null'][testing_df['Null']>0] = 0

/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Missing in FinelineNumber

In [80]:
train_df['FinelineNumber'].fillna(9999,inplace=True)
train_fl_null = train_df[train_df['FinelineNumber']==9999]
train_fl_null = train_fl_null.groupby('VisitNumber',as_index=False)['ScanCount'].count()
train_fl_null.rename(columns={'ScanCount':'Missing'},inplace=True)
train_fl_null.tail()

,VisitNumber,Missing
2749,190230,1
2750,190309,1
2751,190408,1
2752,190651,1
2753,191080,3


In [81]:
testing_df = testing_df.merge(train_fl_null,how='left',on='VisitNumber',copy=True)
testing_df['Missing'].fillna(1,inplace=True)
testing_df['Missing'][testing_df['Missing']>0] = 0

/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## train_fl_num
- Give information about Number
    - DD
    - FL
    - Upc
    - Ratio 

In [82]:
train_fl_num = train_df.groupby(['VisitNumber','FinelineNumber'],as_index=False)['ScanCount'].count()
train_fl_num = train_fl_num.groupby('VisitNumber',as_index=False)['ScanCount'].count()
train_fl_num.rename(columns={'ScanCount': 'N_Fineline'}, inplace=True) 

train_dd_num = train_df.groupby(['VisitNumber','DepartmentDescription'],as_index=False)['ScanCount'].count()
train_dd_num = train_dd_num.groupby('VisitNumber',as_index=False)['ScanCount'].count()
train_dd_num.rename(columns={'ScanCount': 'N_DepartmentDescription'}, inplace=True) 

train_upc_num = train_df.groupby(['VisitNumber','Upc'],as_index=False)['ScanCount'].count()
train_upc_num = train_upc_num.groupby('VisitNumber',as_index=False)['ScanCount'].count()
train_upc_num.rename(columns={'ScanCount': 'N_Upc'}, inplace=True) 

In [83]:
train_fl_num = train_fl_num.merge(train_dd_num,on='VisitNumber')
train_Num = train_fl_num.merge(train_upc_num,on='VisitNumber')

In [84]:
train_Num['Ratio_F_D'] = train_Num['N_Fineline'] / train_Num['N_DepartmentDescription']
train_Num['Ratio_U_D'] = train_Num['N_Upc'] / train_Num['N_DepartmentDescription']

In [85]:
train_Num.shape

(94247, 6)

In [86]:
testing_df = testing_df.merge(train_Num,on='VisitNumber')

## train_sc_sum 
- ScanCount sum by groupby VisitNumber

In [87]:
train_sc_sum = train_df.groupby('VisitNumber',as_index=False)['ScanCount'].sum()
train_sc_sum.rename(columns={'ScanCount':'Sum_sc'},inplace=True)
train_sc_sum.tail()

,VisitNumber,Sum_sc
95669,191343,9
95670,191344,5
95671,191345,17
95672,191346,17
95673,191347,2


In [88]:
testing_df = testing_df.merge(train_sc_sum,on='VisitNumber')

## Encoding the Upc column to Company column

In [89]:
from Unique_for_VisitNumber_and_for_TripType import *
testing_df = decodeStuffNeedsToBeDecoded(testing_df)

## Drop the Upc Column

In [90]:
testing_df.drop(columns='Upc',inplace=True)
testing_df.tail()

,TripType,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,Null,Missing,N_Fineline,N_DepartmentDescription,N_Upc,Ratio_F_D,Ratio_U_D,Sum_sc,Company
644680,39,191346,Sunday,1,PHARMACY OTC,1118,0.0,0.0,16,8,17,2.0,2.125,17,323900
644681,39,191346,Sunday,1,FROZEN FOODS,1752,0.0,0.0,16,8,17,2.0,2.125,17,078742
644682,39,191346,Sunday,1,PRODUCE,4170,0.0,0.0,16,8,17,2.0,2.125,17,40
644683,8,191347,Sunday,1,DAIRY,1512,0.0,0.0,2,2,2,1.0,1.000,2,041900
644684,8,191347,Sunday,1,GROCERY DRY GOODS,3600,0.0,0.0,2,2,2,1.0,1.000,2,038000


## Make DepartmentDescription Column into Dummies

In [91]:
from awesome_functions import *
testing_df = make_df_we_wanted(testing_df,testing_df,test,is_need_null_column=True)

##### Dummy타입으로 만든 컬럼 명 : DepartmentDescription

##### ScanCount는 양수만 사용

##### Train df 만드는 중

##### Null 컬럼을 만듬

> 위 정보들을 Display하는 이유는 이번 FeatureMatrix를 사용한 모델에 Fit할 Test 모델 만들 때 같은 전처리를 하기 위해서다.

In [92]:
testing_df.tail()

,VisitNumber,TripType,Null,Missing,N_Fineline,N_DepartmentDescription,N_Upc,Ratio_F_D,Ratio_U_D,Sum_sc,...,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
94242,191343,25,0,0,35,21,49,11.6667,16.3333,63,...,0,0,0,0,0,0,0,0,0,1
94243,191344,22,0,0,15,10,25,7.5,12.5,25,...,0,0,1,0,0,0,0,0,0,1
94244,191345,39,0,0,156,104,169,19.5,21.125,221,...,0,0,0,0,0,0,0,0,0,1
94245,191346,39,0,0,272,136,289,34,36.125,289,...,0,0,0,0,0,0,0,0,0,1
94246,191347,8,0,0,4,4,4,2,2,4,...,0,0,0,0,0,0,0,0,0,1


In [93]:
testing_df.drop(columns='Company',inplace=True)

## Make Input and Output DataFrame

In [94]:
train_X , train_y = get_df_to_fit(testing_df)

In [95]:
train_X.tail()

,Null,Missing,N_Fineline,N_DepartmentDescription,N_Upc,Ratio_F_D,Ratio_U_D,Sum_sc,Return,1-HR PHOTO,...,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
94242,0,0,35,21,49,11.6667,16.3333,63,0,0,...,0,0,0,0,0,0,0,0,0,1
94243,0,0,15,10,25,7.5,12.5,25,0,0,...,0,0,1,0,0,0,0,0,0,1
94244,0,0,156,104,169,19.5,21.125,221,0,0,...,0,0,0,0,0,0,0,0,0,1
94245,0,0,272,136,289,34,36.125,289,0,0,...,0,0,0,0,0,0,0,0,0,1
94246,0,0,4,4,4,2,2,4,0,0,...,0,0,0,0,0,0,0,0,0,1


In [97]:
train_y = train_y.astype('int')

# Modeling

### Multinomial

In [98]:
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB().fit(train_X,train_y)

### LogisticRegression

In [100]:
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression().fit(train_X,train_y)

In [101]:
from sklearn.metrics import *
print('Multinomial score: ',accuracy_score(train_y,model1.predict(train_X))),
print('Logistic score: ',accuracy_score(train_y,model2.predict(train_X)))

Multinomial score:  0.5798062537799612
Logistic score:  0.5517841416702919


## Tree

In [109]:
from sklearn.tree import  ExtraTreeClassifier, DecisionTreeClassifier
tree1 = DecisionTreeClassifier(max_depth=17).fit(train_X,train_y)
tree2 = ExtraTreeClassifier(max_depth=17).fit(train_X,train_y)

In [110]:
from sklearn.metrics import *
print(' tree score: ',accuracy_score(train_y,tree1.predict(train_X))),
print('Extra tree score: ',accuracy_score(train_y,tree2.predict(train_X)))

 tree score:  0.6531030165416406
Extra tree score:  0.39519560304306767


## Xgboost

In [111]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [112]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder().fit(train_y)
y_labeled = label_enc.transform(train_y)

In [113]:
X_train, X_test, y_train, y_test = train_test_split(train_X, y_labeled, random_state=0)

In [114]:
dtrain = xgb.DMatrix(X_train.values, label=y_train)
dtest = xgb.DMatrix(X_test.values, label=y_test)

In [115]:
num_boost_round = 300
params = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class':38, 
          'max_delta_step': 3, 
          'eta': 0.2}

evals = [(dtrain, 'train'), (dtest, 'eval')]

bst = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
                early_stopping_rounds=10,)

[0]	train-mlogloss:3.07184	eval-mlogloss:3.08453
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:2.5683	eval-mlogloss:2.59522
[2]	train-mlogloss:2.17616	eval-mlogloss:2.21722
[3]	train-mlogloss:1.895	eval-mlogloss:1.94802
[4]	train-mlogloss:1.70389	eval-mlogloss:1.76676
[5]	train-mlogloss:1.56418	eval-mlogloss:1.63498
[6]	train-mlogloss:1.45392	eval-mlogloss:1.53109
[7]	train-mlogloss:1.36255	eval-mlogloss:1.44663
[8]	train-mlogloss:1.28795	eval-mlogloss:1.3778
[9]	train-mlogloss:1.22559	eval-mlogloss:1.32041
[10]	train-mlogloss:1.17168	eval-mlogloss:1.27147
[11]	train-mlogloss:1.12613	eval-mlogloss:1.23041
[12]	train-mlogloss:1.08696	eval-mlogloss:1.19561
[13]	train-mlogloss:1.05279	eval-mlogloss:1.16583
[14]	train-mlogloss:1.0218	eval-mlogloss:1.13883
[15]	train-mlogloss:0.994907	eval-mlogloss:1.11593
[16]	train-mlogloss:0.970902	eval-mlogloss:1.09574
[17]	train-m

In [ ]:
def Fineline_Count(data, fineline_list):
    
    new_data=data[['VisitNumber', 'Weekday']].drop_duplicates()
    for i in range(len(fineline_list)):
        df=data[data['FinelineNumber']==fineline_list[i]]
        df = df.groupby(['VisitNumber'], as_index=False)['Count'].sum()
        df.rename(columns={'Count': 'F_%s' % (i)}, inplace=True)
        new_data = new_data.merge(df, how='left', on=['VisitNumber'], copy=True)
        new_data['F_%s' % (i)].fillna(value=0, inplace=True)
    new_data.drop('Weekday', axis=1, inplace=True)    
    return new_data 

def Department_Count_Products(data, department_list):
    
    new_data=data[['VisitNumber', 'Weekday', 'Sum']].drop_duplicates()
    for i in range(len(department_list)):
       df=data[data['DepartmentDescription']==department_list[i]]
       df = df.groupby(['VisitNumber'], as_index=False)['Count'].sum()
       df.rename(columns={'Count': 'D_%s' % (i)}, inplace=True)
       new_data = new_data.merge(df, how='left', on=['VisitNumber'], copy=True)
       new_data['D_%s' % (i)].fillna(value=0, inplace=True)
       new_data['Ratio_%s' % (i)] = new_data['D_%s' % (i)]/new_data['Sum']
       new_data['Ratio_%s' % (i)].fillna(value=0, inplace=True)
    new_data.drop(['Sum', 'Weekday'], axis=1, inplace=True)
    return new_data
    
def Department_Counts_Neg_Products(data, department_list):

    new_data=data[['VisitNumber', 'Weekday']].drop_duplicates()
    for i in range(len(department_list)):
       df=data[(data['DepartmentDescription']==department_list[i]) & (data['ScanCount']<0)]
       df = df.groupby(['VisitNumber'], as_index=False)['ScanCount'].sum()
       df['ScanCount'] = df['ScanCount']*(-1)
       df.rename(columns={'ScanCount': 'D1_%s' % (i)}, inplace=True)
       new_data = new_data.merge(df, how='left', on=['VisitNumber'], copy=True)
       new_data['D1_%s' % (i)].fillna(value=0, inplace=True)
    new_data.drop('Weekday', axis=1, inplace=True)   
    return new_data 
    
def Department_Counts_Multiple_Products(data, department_list):

    new_data=data[['VisitNumber', 'Weekday']].drop_duplicates()  
    for i in range(len(department_list)):
       df=data[(data['DepartmentDescription']==department_list[i]) & (data['ScanCount']>1)]
       df = df.groupby(['VisitNumber'], as_index=False)['ScanCount'].count()
       df.rename(columns={'ScanCount': 'D2_%s' % (i)}, inplace=True)
       new_data = new_data.merge(df, how='left', on=['VisitNumber'], copy=True)
       new_data['D2_%s' % (i)].fillna(value=0, inplace=True)
    new_data.drop('Weekday', axis=1, inplace=True)   
    return new_data 
    
def Department_Counts_Multiple_Rows(data, department_list):

    new_data=data[['VisitNumber', 'Weekday']].drop_duplicates()
    for i in range(len(department_list)):
       df=data[data['DepartmentDescription']==department_list[i]]
       df = df.groupby(['VisitNumber', 'FinelineNumber'], as_index=False)['Upc'].count()
       df[df['Upc']==1]=0
       df = df.groupby(['VisitNumber'], as_index=False)['Upc'].sum()
       df.rename(columns={'Upc': 'D3_%s' % (i)}, inplace=True)
       new_data = new_data.merge(df, how='left', on=['VisitNumber'], copy=True)
       new_data['D3_%s' % (i)].fillna(value=0, inplace=True)
    new_data.drop('Weekday', axis=1, inplace=True)   
    return new_data 

def Department_Counts_Rows_Pos_ScanCount(data, department_list):

    new_data=data[['VisitNumber', 'Weekday']].drop_duplicates()
    for i in range(len(department_list)):
       df=data[(data['DepartmentDescription']==department_list[i]) & (data['ScanCount']>0)]
       df = df.groupby(['VisitNumber'], as_index=False)['ScanCount'].count()
       df = df.groupby(['VisitNumber'], as_index=False)['ScanCount'].sum()
       df.rename(columns={'ScanCount': 'D4_%s' % (i)}, inplace=True)
       new_data = new_data.merge(df, how='left', on=['VisitNumber'], copy=True)
       new_data['D4_%s' % (i)].fillna(value=0, inplace=True)
    new_data.drop('Weekday', axis=1, inplace=True)   
    return new_data 

def Department_Counts_Rows_Neg_Scancount(data, department_list):

    new_data=data[['VisitNumber', 'Weekday']].drop_duplicates()
    for i in range(len(department_list)):
       df=data[(data['DepartmentDescription']==department_list[i]) & (data['ScanCount']<0)]
       df = df.groupby(['VisitNumber'], as_index=False)['Upc'].count()
       df = df.groupby(['VisitNumber'], as_index=False)['Upc'].sum()
       df.rename(columns={'Upc': 'D5_%s' % (i)}, inplace=True)
       new_data = new_data.merge(df, how='left', on=['VisitNumber'], copy=True)
       new_data['D5_%s' % (i)].fillna(value=0, inplace=True)
    new_data.drop('Weekday', axis=1, inplace=True)   
    return new_data 